In [3]:
import math, pysal, random, shutil
import numpy as np
import csv
import pandas as pd
import geopandas as gp
import csvkit as ck
import itertools

%pylab inline
#%matplotlib inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
regpairlist = pd.read_csv("C:/Lorraine/DATA/REGIONS/pairlist_regions20k.csv")
wardreg = pd.read_csv("C:/Lorraine/DATA/REGIONS/reg_output20k.csv")
OFout = pd.read_csv("C:/Lorraine/DATA/REGIONS/O_Flows_Out20k.csv")
DFin = pd.read_csv("C:/Lorraine/DATA/REGIONS/D_Flows_In20k.csv")
#print regpairlist.head()

#some orig-dest pairs are the same. The first row of the same pair is correct. The second set the index is switched. 
#    origin  destination  flow  OflowsOut  DflowsIn
#0       1            1   716       2819      2031
#1       1            2     6       2819      1166
#bottom row should be 2, 1 for orig, dest. All flow values are correct

#then where origsameid and destsameid are both = TRUE then switch orig and dest ids
#this step is due to the repetition of orig dest ids. The second repeated matching pair should be switched so that the pairs are all unique.

#check to see if the orig and destination id's have changed from the previous row using .shift
regpairlist['origsameid'] = regpairlist.origin == regpairlist.origin.shift(1)
regpairlist['destsameid'] = regpairlist.destination == regpairlist.destination.shift(1)
#find the rows which need swapping
#print regpairlist['origsameid'].dtypes
#convert boolean true/false to int
regpairlist['origsameid'] = regpairlist['origsameid'].astype(int)
regpairlist['destsameid'] = regpairlist['destsameid'].astype(int)
regpairlist['eq'] = regpairlist.apply(lambda row: row['origsameid'] == 1 and row['destsameid'] ==1, axis=1).astype(int)
#print RegOutInflows.head()
s = regpairlist['eq'] ==1
regpairlist.loc[s, ['origin', 'destination']] = regpairlist.loc[s, ['destination','origin']].values
#print regpairlist.head()

wardshapefile = r"C:/Lorraine/DATA/REGIONS/NIW01_sort.shp"
#wards = gp.read_file(wards)
wards = gp.read_file(wardshapefile)
wards = wards.to_crs(epsg=29902)

regression_shapes = wards.merge(wardreg, on='LABEL')
#print regression_shapes.head()
#dissolve geometry by reg id
regions = regression_shapes.dissolve(by='REG_ID')
print len(regions)

#generate centroids
regions['centroid_column'] = regions.centroid

#generate area
regions['area'] = regions.area / 10**6 ##convert from m to km

reg_centroids = regions.loc[:,'centroid_column']
#print reg_centroids.head()

reg_area = regions.loc[:,'area']

#separate centroids into x and y columns
#reg_centroids.str.strip('()').str.split(', ', expand=True).rename(columns={0:'X', 1:'Y'})
regions["x"] = regions.centroid.map(lambda p: p.x)
regions["y"] = regions.centroid.map(lambda p: p.y)
xy = regions.loc[:,'x':'y']
x = regions.loc[:, 'x']
y = regions.loc[:, 'y']
xy['origindex'] = xy.index
xy['destindex'] = xy.index
xyorig = pd.DataFrame(xy)
xyorig = xyorig.rename(columns = {'x':'origx'})
xyorig = xyorig.rename(columns = {'y':'origy'})

#print xyorig

xydest = pd.DataFrame(xy)
xydest = xydest.rename(columns = {'x': 'destx'})
xydest = xydest.rename(columns = {'y': 'desty'})

#print xydest

RegOutflows = regpairlist.merge(OFout[['origin','OflowsOut']], how='left', on='origin') 
RegOutInflows = RegOutflows.merge(DFin[['destination','DflowsIn']], how='left', on='destination') 

##print RegOutInflows.head()

#merge x and y of centroids
##problem lines 
RegOutInflowsxyo = pd.merge(RegOutInflows, xyorig, left_on=['origin'], right_on=['origindex'], how='inner')
#print RegOutInflowsxyo
#RegOutInflowsxyo.reset_index()
RegOutInflowsxy = pd.merge(RegOutInflowsxyo, xydest, left_on=['destination'], right_on=['destindex'], how='inner')
print RegOutInflowsxy.head()

#calculate distances
import math
pd.options.mode.chained_assignment = None  # default='warn'
def calculateDistance(x1,y1,x2,y2):
    #changed from math.sqrt
    dist = np.sqrt((x2-x1)**2 + (y2-y1)**2)
    return dist

RegOutInflowsxy['distance'] = RegOutInflowsxy.apply(lambda x:calculateDistance(x.loc['origx'], x.loc['origy'], x.loc['destx'], x.loc['desty']), axis=1) 

RegFlows = RegOutInflowsxy[['origin', 'destination', 'flow', 'OflowsOut', 'DflowsIn', 'distance', 'origx', 'origy', 'destx', 'desty']]
##print RegFlows.head() 

wardvarsOrig = pd.read_csv("C:/Lorraine/DATA/WARD_DATA/2001/Variables/Ward01_variablesOrig.csv")
wardvarsDest = pd.read_csv("C:/Lorraine/DATA/WARD_DATA/2001/Variables/Ward01_variablesDest.csv")

#Merge the region ids with the variable data

wardflowVarsOrig = wardvarsOrig.merge(wardreg, how='left', on=['LABEL'], left_index=True)
wardflowVarsDest = wardvarsDest.merge(wardreg, how='left', on=['LABEL'], left_index=True)

#print wardflowVarsOrig.head()

#O_Jobs_2001, O_Jobs_2011, O_All persons are all summed by REG_ID, the other columns are percentages so calculate the mean

#wardO = wardflowVarsOrig.groupby('REG_ID').mean().add_prefix('totalO_').reset_index()
#wardD = wardflowVarsDest.groupby('REG_ID').mean().add_prefix('totalD_').reset_index


wardO = wardflowVarsOrig.groupby('REG_ID').agg({'O_Jobs_2001': 'sum', 'O_Jobs_2011': 'sum', 'O_All persons': 'sum', 
                                                'O_CB_PC': 'mean', 'O_ECON_ACT_P': 'mean', 'O_POPDENS_01': 'mean', 'O_MDM_2005': 'mean', 'O_PerH_NoCar': 'mean', 'O_Per_NSEC1-3': 'mean', 'O_RdLenEC_M': 'mean'}).reset_index()
wardO['origin'] = wardO['REG_ID']
##print wardO.head()

wardD = wardflowVarsDest.groupby('REG_ID').agg({'D_Jobs_2001': 'sum', 'D_Jobs_2011': 'sum', 'D_All persons': 'sum', 
                                                'D_CB_PC': 'mean', 'D_ECON_ACT_P': 'mean', 'D_POPDENS_01': 'mean', 'D_MDM_2005': 'mean', 'D_PerH_NoCar': 'mean', 'D_Per_NSEC1-3': 'mean', 'D_RdLenEC_M': 'mean'}).reset_index()
wardD['destination'] = wardD['REG_ID']
##print wardD.head()

wardflowVarsO = RegFlows.merge(wardO, how='left', on=['origin'], left_index=True)
wardflowVarsOD = wardflowVarsO.merge(wardD, how='left', on=['destination'], left_index=True)

##print wardflowVarsOD.head()

#wardflowVarsOD.to_csv('C:/Lorraine/DATA/REGRESS/exploratory_regression/ward01_reg20k_ODvars.csv')
wardflowVarsOD.to_csv('C:/Lorraine/DATA/WARD_DATA/2001/OUTPUT_FILES/ward01_reg20k_ODvars_ADDITIONAL.csv')


25
   origin  destination   flow  origsameid  destsameid  eq  OflowsOut  \
0       1            1  22864           0           0   0      39424   
1       2            1     12           1           1   1      29872   
2       3            1    307           1           1   1      28897   
3       4            1     27           1           1   1      24473   
4       5            1     39           1           1   1      27853   

   DflowsIn          origx          origy  origindex_x  destindex_x  \
0     29920  307986.129490  420813.415025            1            1   
1     29920  242589.692587  379059.151744            2            2   
2     29920  331892.235605  383932.962812            3            3   
3     29920  298251.530335  356171.936801            4            4   
4     29920  339727.174848  335954.301790            5            5   

          destx          desty  origindex_y  destindex_y  
0  307986.12949  420813.415025            1            1  
1  307986.12949  42